
Trained:
    5010 sentences

Notes:
    Multiple Animation: Ang bata ang kumakanta (habang)[trigger word for multiple animation] naglalakad


# Removing non-alphabet characters (numerical and special characters)

In [1]:
import os
import string
import data_utils
import timeit

raw_output = r'dataset/tl-storytelling-dataset.txt'
processed_output = r'dataset/processed/tl-storytelling-dataset-processed.txt'
final_processed_output = r'dataset/processed/tl-storytelling-dataset-processed-final.txt'

sentence_list = data_utils.read_text_file(raw_output)
clean_sentence_list = []
for sentence in sentence_list:
    clean_sentence_list.append(' '.join(e for e in sentence.split() if e.isalpha()))
    
data_utils.write_text_file(processed_output, clean_sentence_list)
print("Preprocessing Step [2/4] Done!")

Preprocessing Step [2/4] Done!


In [9]:
# Always look for this � or make sure encoding='utf-8'
total_words = sum([len(sentence.split()) for sentence in clean_sentence_list])
total_sentence = len(clean_sentence_list)
unique_words = []

for sentence in clean_sentence_list:
    for word in sentence.split():
        word = word.lower()
        if word not in unique_words:
            unique_words.append(word)
print("before:", "tokens:", total_words, "sentence:", total_sentence, 'unique_words:', len(unique_words))

before: tokens: 45792 sentence: 5005 unique_words: 5646


# Comparing the before and after removing stop-words and empty array

In [2]:
# Always look for this � or make sure encoding='utf-8'
total_words = sum([len(sentence.split()) for sentence in clean_sentence_list])
total_sentence = len(clean_sentence_list)
print("before:", "words:", total_words, "sentence:", total_sentence)

#Modified stopword to include some number and weather (e.g. isa, araw)
clean_sentence_list = data_utils.text_to_list_without_stopwords(processed_output)
clean_sentence_list = list(filter(None, clean_sentence_list)) #Removing empty array
clean_sentence_list = [' '.join(sentence) for sentence in clean_sentence_list]
total_words = sum([len(sentence.split()) for sentence in clean_sentence_list])
total_sentence = len(clean_sentence_list)
print("after:", "words:", total_words, "sentence:", total_sentence)


data_utils.write_text_file(final_processed_output, clean_sentence_list)
print("Preprocessing Step [3/4] Done!")

before: words: 45792 sentence: 5005
after: words: 28676 sentence: 4527
Preprocessing Step [3/4] Done!


# Automatic POS Tagging 

## Filipino Standford POS Tagger (FSPOST) - Nicco Noccon (2020)

In [ ]:
import fspost
fspost.set_java_path('')

# Start automatic POS Tagging annotation
clean_special_numeric_stopword = data_utils.read_text_file(final_processed_output)
tagged_list = fspost.tag_string_list(clean_special_numeric_stopword)

## FSPOST with Morphinas Stemmer (POS Tag with Stemming)

In [20]:
import subprocess
import os

def fspost_lemmatize(sentence):
    p1 = subprocess.run(['java', '-jar', '--enable-preview', 'fspost-lemma.jar', sentence], capture_output=True, text=True)
    output = p1.stdout.split('\n')[3:-1]
    tag = output[0].split()
    word = output[1].split()
    return [(word,tag) for word,tag in zip(word,tag) if word != '?']

progress_ctr = 0
fspostagged_path = r'dataset\annotation\tl-storytelling-pos-tagged-1.txt'
sentence_list = data_utils.read_text_file(fspostagged_path)     
tagged_list = []  # Initialize an empty list

for sentence in sentence_list:
    tagged_tuple = fspost_lemmatize(sentence)  # Tag each sentence in the list
    tagged_list.append(tagged_tuple)  # Insert tagged sentence in the new list
    progress_ctr += 1
    print(progress_ctr, "/", len(sentence_list))  # Progress Counter
    print("Before: ", sentence)
    print("After: ",tagged_tuple)

1 / 4523
Before:  alitaptap alitaptap halina halina mga matang kumikindat halina kumislap dilim
After:  [('alitaptap', 'FW'), ('alitaptap', 'FW'), ('halina', 'FW'), ('halina', 'FW'), ('mga', 'DTCP'), ('mata', 'NNC_CCP'), ('kindat', 'VBTR'), ('halina', 'NNC'), ('kislap', 'VBAF'), ('dilim', 'NNC')]
2 / 4523
Before:  alitaptap alitaptap halina halina araw sisikat
After:  [('alitaptap', 'FW'), ('alitaptap', 'FW'), ('halina', 'FW'), ('halina', 'FW'), ('araw', 'RBW'), ('sikat', 'VBTF')]
3 / 4523
Before:  darating umaga
After:  [('darating', 'VBTF'), ('umaga', 'RBW')]
4 / 4523
Before:  halina huwag matakot dilim alitaptap alitaptap halina halina
After:  [('halina', 'RBW'), ('huwag', 'RBF'), ('takot', 'VBW'), ('dilim', 'NNC'), ('alitaptap', 'NNC'), ('alitaptap', 'NNC'), ('halina', 'NNC'), ('halina', 'NNC')]
5 / 4523
Before:  halina aliwin
After:  [('halina', 'RBW'), ('aliw', 'NNC')]
6 / 4523
Before:  nabalot dilim
After:  [('balot', 'VBTS'), ('dilim', 'NNC')]
7 / 4523
Before:  mga matang malun

# Saving to CSV file

In [21]:
import numpy as np
import pandas as pd

# Sample Text File
# [
#     [('alitaptap', 'FW'), ('alitaptap', 'FW'), ('halina', 'FW'), ('halina', 'FW'), ('ng', 'CCB'), ('mga', 'DTCP'), ('matang', 'NNC_CCP'), ('kumikindat', 'VBTR'), ('halina', 'NNC'), ('kumislap', 'VBAF'), ('dilim', 'NNC')],
#     [('test', 'FW'), ('alitaptap', 'FW'), ('halina', 'FW'), ('halina', 'FW'), ('araw', 'RBW'), ('sisikat', 'VBTF')]
# ]

temp_batch = tagged_list
dataset = []
for subset in temp_batch:
    dataset.append(list(map(list, zip(*subset))))
df = pd.DataFrame(dataset, columns=['tokens','tags'])

df.to_csv(r'dataset/annotation/tl-storytelling-pos-tagged-lemmatized.csv', index=False)
print("Preprocessing Step [4/4] Done!")
df.head()

Preprocessing Step [4/4] Done!


,tokens,tags
0,"[alitaptap, alitaptap, halina, halina, mga, ma...","[FW, FW, FW, FW, DTCP, NNC_CCP, VBTR, NNC, VBA..."
1,"[alitaptap, alitaptap, halina, halina, araw, s...","[FW, FW, FW, FW, RBW, VBTF]"
2,"[darating, umaga]","[VBTF, RBW]"
3,"[halina, huwag, takot, dilim, alitaptap, alita...","[RBW, RBF, VBW, NNC, NNC, NNC, NNC, NNC]"
4,"[halina, aliw]","[RBW, NNC]"


# Saving to txt file (words only)

In [22]:
path_pos_w = r'dataset/annotation/tl-storytelling-pos-tagged-lemmatized.txt'
file_pos_w = open(path_pos_w, 'a+', encoding='utf-8')

for batch in tagged_list:
    pos_w = []
    for tags in batch:
        token_tag = tags[-1]
        token = tags[0]
        if len(token) > 1:
            # JJD, 
            if 'NN' in token_tag or 'FW' in token_tag or 'TS' in token_tag or\
                'VB' in token_tag or \
                'JJD' in token_tag:
                pos_w.append(token)

    if pos_w != []:
        file_pos_w.write(' '.join(pos_w) + '\n')
        
file_pos_w.close()

# Separating Part-of-Speech Tags (Optional)

In [41]:
path_nn_w = r'dataset/annotation/tl-noun-word.txt'
path_adj_w = r'dataset/annotation/tl-adj-word.txt'
path_pos_w = r'dataset/annotation/tl-pos-word.txt'

file_nn_w = open(path_nn_w, 'a+', encoding='utf-8')
file_adj_w = open(path_adj_w, 'a+', encoding='utf-8')
file_vb_w = open(path_pos_w, 'a+', encoding='utf-8')

# Remove comments to include tag.txt

# path_nn_t = r'dataset/annotation/tl-noun-tag.txt'
# path_adj_t = r'dataset/annotation/tl-adj-tag.txt'
# path_vb_t = r'dataset/annotation/tl-verb-tag.txt'

# file_nn_t = open(path_nn_t, 'a+', encoding='utf-8')
# file_adj_t = open(path_adj_t, 'a+', encoding='utf-8')
# file_vb_t = open(path_vb_t, 'a+', encoding='utf-8')

for batch in batch1:
    nn_w = []
    adj_w = []
    vb_w = []

    nn_t = []
    adj_t = []
    vb_t = []
    for tags in batch:
        token_tag = tags[-1]
        token = tags[0]
        if len(token) > 1:
            if 'NN' in token_tag or 'FW' in token_tag :
                nn_w.append(token)
                nn_t.append(token_tag)
            if 'VB' in token_tag:
                vb_w.append(token)
                vb_t.append(token_tag)
            if 'JJ' in token_tag or 'RB' in token_tag or 'DT' in token_tag or 'CD' in token_tag or 'TS' in token_tag or 'PR' in token_tag:
                adj_w.append(token)
                adj_t.append(token_tag)

    if nn_w != []:
        file_nn_w.write(' '.join(nn_w) + '\n')
    if adj_w != []:
        file_adj_w.write(' '.join(adj_w) + '\n')
    if vb_w != []:
        file_vb_w.write(' '.join(vb_w) + '\n')

    # if nn_t != []:
    #     file_nn_t.write(' '.join(nn_t) + '\n')
    # if adj_t != []:
    #     file_adj_t.write(' '.join(adj_t) + '\n')
    # if vb_t != []:
    #     file_vb_t.write(' '.join(vb_t) + '\n')
        
file_nn_w.close()
file_adj_w.close()
file_vb_w.close()

# file_nn_t.close()
# file_adj_t.close()
# file_vb_t.close()